In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Columns: 60485 entries, id to ENSG00000181518.3
dtypes: float64(55695), int64(4789), object(1)
memory usage: 128.7+ MB


In [4]:
data.head()

,id,Label,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-2V-A95S-01A,-1,1.322600,0.000000,2.60713,0,1.411440,3.27352,0,0.006410,...,0,2.710970,0.0,0.435580,0.000000,3.90541,0.017601,0,1.708020,0.0
1,TCGA-2Y-A9GU-01A,1,0.000000,0.006431,1.49737,0,0.598584,2.39426,0,0.006599,...,0,0.812853,0.0,0.208534,0.147081,4.05334,0.368102,0,0.182464,0.0
2,TCGA-2Y-A9GV-01A,1,0.000000,0.000000,1.28544,0,1.366550,2.36780,0,0.005341,...,0,6.645780,0.0,0.170978,0.176644,2.90339,0.262281,0,0.386022,0.0
3,TCGA-2Y-A9GX-01A,1,0.201191,0.000000,1.45312,0,1.566180,2.84619,0,0.006135,...,0,5.085340,0.0,0.067894,0.000000,3.70349,0.422939,0,0.666707,0.0
4,TCGA-2Y-A9GY-01A,-1,0.043040,0.000000,2.25014,0,1.421440,2.73472,0,0.024669,...,0,0.140969,0.0,0.370195,0.000000,3.85624,0.236684,0,0.513950,0.0


In [5]:
X = data.drop(['id', 'Label'], axis=1, inplace=False)
y = data['Label']

In [6]:
print(X.shape)
print(y.shape)

(279, 60483)
(279,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.3)

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(195, 60483)
(195,)
(84, 60483)
(84,)


In [9]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [10]:
# Create an SelectKBest object to select features with two best ANOVA F-Values
fvalue_selector = SelectKBest(f_classif, k=50)

# Apply the SelectKBest object to the features and target
X_kbest_train = fvalue_selector.fit_transform(X_train, y_train)

C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [    3     6    12 ... 60462 60473 60480] are constant.
  UserWarning)
C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [11]:
# Create an SelectKBest object to select features with two best ANOVA F-Values
fvalue_selector = SelectKBest(f_classif, k=50)

# Apply the SelectKBest object to the features and target
X_kbest_test = fvalue_selector.fit_transform(X_test, y_test)

C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [    3     6    12 ... 60473 60480 60482] are constant.
  UserWarning)
C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [12]:
from sklearn.model_selection import GridSearchCV

In [13]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {'n_neighbors': range(5, 100, 2)}

grid = GridSearchCV(KNeighborsClassifier(), param_grid)
grid.fit(X_kbest_train, y_train)

C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': range(5, 100, 2)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [14]:
grid_predictions = grid.score(X_kbest_test, y_test)
grid_predictions

0.5

In [15]:
grid.best_params_

{'n_neighbors': 7}

In [18]:
clf = KNeighborsClassifier(n_neighbors=6)
clf.fit(X_kbest_test, y_test)
clf.score(X_kbest_test, y_test)

0.7976190476190477

In [19]:
test = pd.read_csv('test.csv')

In [20]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Columns: 60484 entries, id to ENSG00000181518.3
dtypes: float64(51458), int64(9026)
memory usage: 32.8 MB


In [21]:
test.head()

,id,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,3001,0.000000,0.006764,1.68576,0.0,1.046760,2.56965,0,0.000000,5.38054,...,0,6.30029,0,0.076595,0.000000,3.02069,0.196880,0,0.538068,0.0
1,3002,0.247055,0.000000,1.89639,0.0,1.996530,2.47638,0,0.016952,5.98824,...,0,5.04563,0,0.137816,0.000000,3.56454,0.397989,0,0.574163,0.0
2,3003,0.026951,0.000000,1.80290,0.0,1.568690,1.90269,0,0.007726,5.97967,...,0,6.00935,0,0.000000,0.044642,3.64769,0.217624,0,0.135791,0.0
3,3004,0.000000,0.000000,1.51205,0.0,0.938359,1.81372,0,0.003923,6.46130,...,0,5.51651,0,0.127467,0.000000,2.88242,0.612775,0,0.807993,0.0
4,3005,0.643548,0.000000,1.35997,0.0,1.254810,3.64841,0,0.128797,6.27939,...,0,2.28184,0,0.058164,0.060170,4.11811,0.150976,0,0.416153,0.0


In [22]:
X_test_new = test.drop(['id'], axis=1, inplace=False)
print(X_test_new.shape)

(71, 60483)


In [23]:
print(X.shape)
print(X_test_new.shape)

(279, 60483)
(71, 60483)


In [24]:
# Create an SelectKBest object to select features with two best ANOVA F-Values
fvalue_selector = SelectKBest(f_classif, k=50)

# Apply the SelectKBest object to the features and target
X_kbest = fvalue_selector.fit_transform(X, y)
X_kbest_test_new = fvalue_selector.transform(X_test_new)

C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [    3     6    12 ... 60462 60473 60480] are constant.
  UserWarning)
C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [27]:
clf = KNeighborsClassifier(n_neighbors=6)
clf.fit(X_kbest, y)
clf.score(X_kbest, y)

0.7096774193548387

In [31]:
result_df = pd.DataFrame(test['id']).join(pd.DataFrame(clf.predict(X_kbest_test_new)))
result_df.columns = ['id','Label']
result_df.to_csv('sample.csv', index=False)